# Neural Netoworks

In [18]:
# Import required packages for this chapter

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix

# Credit Card Use

Consider the hypothetical bank data in on consumers’ use of credit card credit facilities. Import dataset _creditcard.csv_. Use Python program to illustrate one pass through a simple neural network (Randomly generate initial weight values)

_Years: number of years the customer has been with the bank_

_Salary: customer’s salary (in thousands of dollars)_

_Used Credit: 1 = customer has left an unpaid credit card balance at the end of at least one month in the prior year, 0 = balance was paid off at the end of each month_

__Data Preprocessing.__ The dataset is too small, we will not split the data. We are using the whole set as a training set to estabish our model.

In [19]:
# Load the data into creditcard_df dataframe
# Use critical functions to explore the dataframe using print() to show results

creditcard_df=pd.read_csv("C:/Users/lisaj/OneDrive/Documents/Documents/MIS 536/Module8Datasets/creditcard.csv")

print(creditcard_df.shape)
print(creditcard_df.count())
print(creditcard_df.describe().transpose())




(6, 3)
Years          6
Salary         6
Used_Credit    6
dtype: int64
             count       mean        std   min    25%   50%    75%    max
Years          6.0   7.833333   6.968979   1.0   3.25   5.0  12.75   18.0
Salary         6.0  76.000000  26.608269  43.0  56.00  76.5  93.25  112.0
Used_Credit    6.0   0.500000   0.547723   0.0   0.00   0.5   1.00    1.0


In [3]:
# construct the dataset: predictors(X) and outcome(y)
# print out results to varify

X=creditcard_df.drop('Used_Credit', axis='columns')
y=creditcard_df['Used_Credit']

print(X, y)

   Years  Salary
0      4      43
1     18      65
2      1      53
3      3      95
4     15      88
5      6     112 0    0
1    1
2    0
3    0
4    1
5    1
Name: Used_Credit, dtype: int64


In [4]:
# Normalize data uisng standard scalar.
# Do we need to standardize both predictors and outcome or just one of thoese two?

scaler=StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

# Both predictors need to be standardized since they are both continuous variables.

In [5]:
# The dataset is too small to partition into training set and validation set. We will use the whole dataset for training
# Multi-layer Perceptron classifier: one layer with 3 hidden nodes, set the activation function to be logistic function, solver to be lbfgs

mlp=MLPClassifier(hidden_layer_sizes=(3), activation='logistic', solver='lbfgs', random_state=1)

mlp.fit(X, y)


MLPClassifier(activation='logistic', hidden_layer_sizes=3, random_state=1,
              solver='lbfgs')

In [6]:
predictions=mlp.predict(X)
print(predictions)

[0 1 0 0 1 1]


In [7]:
print(confusion_matrix(y, predictions))
print(classification_report(y, predictions))

[[3 0]
 [0 3]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         3

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6



In [8]:
# Network structure
# Hint: sample code given in in class exercise

for i, (weights, intercepts) in enumerate(zip(mlp.coefs_, mlp.intercepts_)):
    print('Hidden layer' if i == 0 else 'Output layer', '{0[0]} => {0[1]}'.format(weights.shape))
    print(' Intercepts:\n ', intercepts)
    print(' Weights:')
    for weight in weights:
        print(' ', weight)
    print()

Hidden layer 2 => 3
 Intercepts:
  [-0.09991553 -0.99226352  2.54413658]
 Weights:
  [-2.79453344  4.31675176 -6.27991604]
  [-1.42873363  2.91069399 -4.7071766 ]

Output layer 3 => 1
 Intercepts:
  [3.19201542]
 Weights:
  [-4.50433939]
  [6.74961315]
  [-8.33092289]



In [9]:
# Predictions: merge predicted classification and the probability to each class into the orignal table

temp=pd.concat([pd.DataFrame(y).reset_index(), pd.DataFrame(predictions, columns=['Predicted'])], axis=1)
predict=pd.concat([temp, pd.DataFrame(mlp.predict_proba(X), columns=[1,2]).reset_index()], axis=1)

print(predict)


   index  Used_Credit  Predicted  index         1         2
0      0            0          0      0  0.999926  0.000074
1      1            1          1      1  0.000058  0.999942
2      2            0          0      2  0.999931  0.000069
3      3            0          0      3  0.999341  0.000659
4      4            1          1      4  0.000055  0.999945
5      5            1          1      5  0.000433  0.999567


In [10]:
# print the confusion_matrix and classification_report

print(confusion_matrix(y, predictions))
print(classification_report(y, predictions))

[[3 0]
 [0 3]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         3

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6



In [11]:
# Interpret your results:

#According to the confusion matrix, three customers had at a credit card balance at the end of at least one month in the prior year. Three customers did not have a balance at the end of the month
#in the previous year on their credit card.

#The classification report indicates that the predictions in the model I created were 100% accurate. 

# Lesson learned from this excercise:

#In this exercise, I learned how to create a multilayer perceptron model,  merge the predicted classification and probability of each class with the original table, and create and 
# interpret a classification matrix and classification report.

# Car Sales

Consider the data on used cars (_toyotacorolla.csv_) with 1436 records and details on 38 attributes, including Price, Age, KM, HP, and other specifcations. The goal is to predict the price of a used Toyota Corolla based on its specifcations.

Use predictors Age_08_04, KM, Fuel_Type, HP, Automatic, Doors, Quarterly_Tax, Mfr_Guarantee, Guarantee_Period, Airco, Automatic_airco, CD_Player, Powered_Windows, Sport_Model, and Tow_Bar.

__Data Preprocessing.__ TUse the scikit-learn transformer _MinMaxScaler()_ to scale the data to the range [0, 1]. Use separate transformer for the input and output data. To create the dummy variables, use the pandas function _pd.get_dummies()_. Partition the data into training (60%) and validation (40%) sets.

## Data Preprocessing

In [20]:
# load the data
# explore the data

toyotacorolla_df=pd.read_csv("C:/Users/lisaj/OneDrive/Documents/Documents/MIS 536/Module8Datasets/toyotacorolla.csv")

print(toyotacorolla_df.shape)
print(toyotacorolla_df.describe().transpose())

print('\n\n', toyotacorolla_df.dtypes)
print('\n\n', toyotacorolla_df.count())


(2872, 39)
                    count          mean           std     min       25%  \
Id                 1436.0    721.555014    416.476890     1.0    361.75   
Price              1436.0  10730.824513   3626.964585  4350.0   8450.00   
Age_08_04          1436.0     55.947075     18.599988     1.0     44.00   
Mfg_Month          1436.0      5.548747      3.354085     1.0      3.00   
Mfg_Year           1436.0   1999.625348      1.540722  1998.0   1998.00   
KM                 1436.0  68533.259749  37506.448872     1.0  43000.00   
HP                 1436.0    101.502089     14.981080    69.0     90.00   
Met_Color          1436.0      0.674791      0.468616     0.0      0.00   
Automatic          1436.0      0.055710      0.229441     0.0      0.00   
CC                 1436.0   1576.855850    424.386770  1300.0   1400.00   
Doors              1436.0      4.033426      0.952677     2.0      3.00   
Cylinders          1436.0      4.000000      0.000000     4.0      4.00   
Gears         

In [21]:
# convert the categorical data Fuel_type into dummy variables

dummy_Fuel_Type=pd.get_dummies(toyotacorolla_df['Fuel_Type'])
print(dummy_Fuel_Type.head(3))

toyotacorolla_df=pd.concat([toyotacorolla_df, dummy_Fuel_Type], axis=1)

toyotacorolla_df=toyotacorolla_df.rename(columns={1: 'CNG', 2: 'Diesel', 3: 'Petrol'})

toyotacorolla_df=toyotacorolla_df.drop(['Fuel_Type'], axis=1)

print(toyotacorolla_df)

   CNG  Diesel  Petrol
0    0       0       0
1    0       1       0
2    0       0       0
          Id                                              Model    Price  \
0        NaN                                                NaN      NaN   
1        1.0      TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  13500.0   
2        NaN                                                NaN      NaN   
3        2.0      TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  13750.0   
4        NaN                                                NaN      NaN   
...      ...                                                ...      ...   
2867  1440.0  TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...   8500.0   
2868     NaN                                                NaN      NaN   
2869  1441.0  TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...   7250.0   
2870     NaN                                                NaN      NaN   
2871  1442.0        TOYOTA Corolla 1.6 LB LINEA TERRA 4/5-Doors   6950.0

In [22]:
# Removing missing values

toyotacorolla_df=toyotacorolla_df.dropna(thresh=len(toyotacorolla_df.columns)-2)
print(toyotacorolla_df)


          Id                                              Model    Price  \
1        1.0      TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  13500.0   
3        2.0      TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  13750.0   
5        3.0      TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  13950.0   
7        4.0      TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors  14950.0   
9        5.0        TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors  13750.0   
...      ...                                                ...      ...   
2863  1438.0         TOYOTA Corolla 1.3 16V HATCHB G6 2/3-Doors   7500.0   
2865  1439.0  TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...  10845.0   
2867  1440.0  TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...   8500.0   
2869  1441.0  TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...   7250.0   
2871  1442.0        TOYOTA Corolla 1.6 LB LINEA TERRA 4/5-Doors   6950.0   

      Age_08_04  Mfg_Month  Mfg_Year       KM     HP  Met_Color   Color  ...  \
1      

In [33]:
# separate out predictors and response variables

X=toyotacorolla_df[['Age_08_04', 'KM', 'CNG', 'Diesel', 'Petrol', 'HP', 'Automatic', 'Doors', 'Quarterly_Tax', 'Mfr_Guarantee', 'Guarantee_Period', 'Airco', 'Automatic_airco',
                   'CD_Player', 'Powered_Windows', 'Sport_Model', 'Tow_Bar']]
y=toyotacorolla_df[['Price']]

# partition data

X_train, X_test, y_training, y_valid=train_test_split(X, y, test_size=0.4, random_state=12)
print(X_train.shape)


(861, 17)


In [45]:
# Do we need to standardize both predictors and outcome or just one of thoese two?
# Both the predictors and outcome need to be standardized. Like the outcome variable, price, many of the predictors are continuous variables.

# Normalizing the outcome variable

scaler=MinMaxScaler()
y=scaler.fit_transform(y)
y_training=scaler.fit_transform(y_training)
y_valid=scaler.fit_transform(y_valid)

# Normalizing the predictors

scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

In [42]:
# train neural network with 2 layers of 5 hidden nodes on each layer
# don't forget to bring y_train back to a single array use dataframename.ravel() in the model fit.

toyotacorolla_df_mlp=MLPRegressor(hidden_layer_sizes=(5, 5), max_iter=500)
toyotacorolla_df_mlp.fit(X_train, y_training.ravel())


MLPRegressor(hidden_layer_sizes=(5, 5), max_iter=500)

In [47]:
# Calculate predictions on validation dataset.
# compute model accuracy using .score()

y_predict=toyotacorolla_df_mlp.predict(X_test)
print(toyotacorolla_df_mlp.score(X_test, y_valid))
print(toyotacorolla_df_mlp.score(X_train, y_training))


-0.6387179459598549
-0.12107967278867449


In [49]:
# Network structure
# Hint: sample code given in in class exercise

for i, (weights, intercepts) in enumerate(zip(toyotacorolla_df_mlp.coefs_, toyotacorolla_df_mlp.intercepts_)):
    print('Hidden layer' if i == 0 else 'Output layer', '{0[0]} => {0[1]}'.format(weights.shape))
    print(' Intercepts:\n ', intercepts)
    print(' Weights:')
    for weight in weights:
        print(' ', weight)
    print()


Hidden layer 17 => 5
 Intercepts:
  [-0.19087548 -0.06302096 -0.44331025  0.02796896 -0.43956634]
 Weights:
  [-0.06686866  0.58084525 -0.12471296 -0.1193471   0.21900539]
  [ 0.14886457  0.27358225 -0.01262114  0.35491901  0.23483677]
  [ 0.57170609 -0.06242711  0.41156168  0.29996076 -0.13904494]
  [ 0.01937211  0.25944857 -0.32927108  0.34061702 -0.17836413]
  [-0.38266025 -0.0135512   0.24216167 -0.04947668  0.21607957]
  [ 0.50830655  0.20874168  0.07193375  0.43968568 -0.23521125]
  [ 0.4757383   0.14742105  0.43925083  0.40365603 -0.14830141]
  [-0.14123879 -0.5070056  -0.28244357  0.24740461 -0.17147276]
  [-0.21583205 -0.49695531  0.20610616 -0.33791636 -0.29920054]
  [-0.14653752  0.34168422 -0.17842453 -0.14076667  0.22009073]
  [ 0.0832011   0.39085617  0.13418838 -0.25518688  0.04039651]
  [-0.03910949 -0.50595025  0.0742757  -0.33641944 -0.06847138]
  [ 0.46734069 -0.04370006  0.35380588 -0.23330786  0.47162621]
  [ 0.14001221 -0.36410391 -0.00801571 -0.2306153  -0.052235

In [50]:
# Can we print out the confusion_matrix and classification_report?
# If you can, show us the print out. If you cannot, can you explain why?

print(confusion_matrix(y_valid, predictions))
print(classification_report(y_valid, predictions))

#No, we cannot print out a confusion matrix and classification report when using a multi-layer perceptron regressor. I believe the reason is because the dependent variable
# is continous and not categorical.

ValueError: Found input variables with inconsistent numbers of samples: [575, 6]

In [ ]:
# Interpret your results:

# The model accuracy is pretty low. The score for the training set is -63, and the score for the validation set is -12. 

# Lesson learned from this excercise:
# I learned created an MLR Regressor so that I could build a neural network. Given the model accuracy score and my lack of expertise on selling used Toyota
# Corollas, it would probably take me awhile to create a better neural network. 

